现在，我们将开始旅程，使用我们刚刚学习的Python知识制作神经网络。  
我们会沿着这个旅程，从简单开始，小步前进，逐步建立Python程序。  
从小处入手，然后让程序慢慢长大，这是构建中等复杂度计算机代码的一种明智的方式。  
在完成了刚才的工作之后，我们非常自然地从建立神经网络类的框架开始，让我们直接前进吧！

# 完整的神经网络代码
这些代码可用于创建、训练和查询3层神经网络，进行几乎任何任务，这么看来，代码不算太多，下一步，我们将进行特定任务，学习识别手写数字。

In [101]:
import numpy
# scipy.special for the sigmoid function expit()
import scipy.special
# library for plotting arrays
import matplotlib.pyplot
# ensure the plots are inside this notebook, not an external window
%matplotlib inline

In [102]:
class neuralNetwork:
    
    # initialise the neural network
    def __init__(self,inputnodes,hiddennodes,outputnodes,learningrate):
        # set number of nodes in each input,hidden,output layer
        self.inodes=inputnodes
        self.hnodes=hiddennodes
        self.onodes=outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc
        self.wih=(numpy.random.rand(self.hnodes,self.inodes)-0.5)
        self.who=(numpy.random.rand(self.onodes,self.hnodes)-0.5)
        
        # learning rate
        self.lr=learningrate
        
        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        pass
    
    # train the neural network
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = numpy.dot(self.who.T, output_errors) 
        
        # update the weights for the links between the hidden and output layers
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        
        pass
    
    # query the neural network
    def query(self,inputs_list):
        # convert inputs list to 2d array
        inputs=numpy.array(inputs_list,ndmin=2).T
        
        # calculate sugnals into hidden layer
        hidden_inputs=numpy.dot(self.wih,inputs)
        # calculate sugnals emerging from hidden layer
        hidden_outputs=self.activation_function(hidden_inputs)
        # calculate sugnals into output layer
        final_inputs=numpy.dot(self.who,hidden_outputs)
        # calculate sugnals emerging from final output layer
        final_outputs=self.activation_function(final_inputs)
        
        return final_outputs

In [103]:
# number of input, hidden and output nodes
input_nodes = 784
hidden_nodes = 200
output_nodes = 10

# learning rate
learning_rate = 0.1

# create instance of neural network
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

In [104]:
# load the mnist training data CSV file into a list
training_data_file = open("mnist_train_100.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [105]:
# train the neural network

# epochs is the number of times the training data set is used for training
epochs = 5

for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        pass
    pass

In [106]:
# load the mnist training data CSV file into a list
test_data_file = open("mnist_test_10.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [107]:
# test the neural network

# scorecard for how well the network performs, initially empty
scorecard = []

# go through all the records in the test data set
for record in test_data_list:
    # split the record by the ',' commas
    all_values = record.split(',')
    # correct answer is first value
    correct_label = int(all_values[0])
    # scale and shift the inputs
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    # query the network
    outputs = n.query(inputs)
    # the index of the highest value corresponds to the label
    label = numpy.argmax(outputs)
    # append correct or incorrect to list
    if (label == correct_label):
        # network's answer matches correct answer, add 1 to scorecard
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        pass
    
    pass

In [114]:
# calculate the performance score, the fraction of correct answers
scorecard_array = numpy.asarray(scorecard)
print ("performance = ", scorecard_array.sum() / scorecard_array.size)

performance =  0.6


# 框架代码  

让我们勾勒神经网络类的大概样子。我们知道，它应该至少有3个函数：  
- 初始化函数——设定输入层节点、隐藏层节点和输出层节点的数量。
- 训练——学习给定训练集样本后，优化权重
- 查询——给定输入，从输出节点给出答案  

目前，这些函数还未完全定义，也许还需要更多的函数，但是，就目前而言，让我们从这些函数起步。  
**所编写的代码框架如下所示：**

```python
# neural network class definition
class neuralNetwork:
    # initialise the neural network
    def __init__(self):
        pass
    # train the neural network
    def train(self):
        pass
    # query the neural network
    def query(self):
        pass
```

事实上这是一个坚实的框架，在这个框架上，你可以充实神经网络工作的详细细节了。

# 初始化网络

从初始化网络开始。我们需要设置输入层节点、隐藏层节点和输出层节点的数量。这些节点数量定义了神经网络的形状和尺寸。我们不会将这些数量固定，而是当我们使用参数创建一个新的神经网络对象时，才会确定这些数量。通过这种方式，我们保留了选择的玉帝，轻松地创建不同大小的新神经网络。  
在我们刚刚所做出的决定中，其底层蕴含着一个重要意义，只要可能就创建一般代码，它迫使我们以一种更深更广泛的使用方式思考求解问题，意味着我们的解决方案可以适用于不同的场景。在此处，这意味着，我们将尽可能地位神经网络开发代码，使神经网络保持尽可能多地开放有用的选项，并将假设降低到最小限度。  
同时也不要忘了学习率。当创建新的神经网络是，这也是待设置的有用参数。让我们看看初始函数是什么样子的：

```python
def __init__(self,inputnodes,hiddennodes,putputnodes,learningrate):
    # set number of nodes in each input,hidden,output layer
    self.inodes=inputnpdes
    self.hnodes=hiddennodes
    self.onodes=outputnodes
    # learning rate
    self.lr=learningrate
    pass
```

让我们使用所定义的神经网络类，尝试创建每层3个节点、学习率为0.5的小型神经网络对象。
```python
# number of input, hidden and output nodes
input_nodes=3
hidden_nodes=3
output_nodes=3

# learning rate is 0.5
learning_rate=0.5

# create instance of neural network
n=neuralNetwork(input_nodes,hidden_nodes,output_nodes,learning_rate
```

# 权重——网络的核心

下一步是创建网络的节点和链接，网络中最重要的部分是链接权重，我们是用这些权重来计算前馈信号、反向传播误差，并且在试图改进网络时优化链接权重本身。  
前面我们看到，可以使用矩阵简明地表示权重。因此，我们可以创建：
- 在输入层和隐藏层之间的链接权重矩阵W (input_hidden，大小为input_nodes乘以hidden_nodes。
- 在隐藏层和输出层之前的链接群众矩阵W（hidden_output,大小为hidden_nodes乘以output_nodes。

``` python
# link weight matrices, wih and who
# weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
# w11 w21
# w12 w22 etc
self.wih=(numpy.random.rand(self.hnodes,self.inodes)-0.5)
self.who=(numpy.random.rand(self.onodes,self.hnodes)-0.5)
```

做得好！我们已经实现了神经网络的心脏——链接权重矩阵！

# 可选项：较复杂的权重

我们可以选择上方这种简单却是很流行的优化初始权重的方式。  
有些人更喜欢稍微复杂的方法来创建初始随机权重。他们使用正太概率分布采样权重，其中平均值为0，标准方差为节点传入链接数目的开方，即$$1/\sqrt{传入链接数目}$$  
在numpy程序哭的帮助下，紫红色是很容易实现的，同样，Google可以帮助我们找到合适的文档。numpy.random.normal()函数可以帮助我们以正态分布的方式采样。由于我们所需要的是随机矩阵，而不是单个数字所以采用分布中心值、标准方差和numpy数组的大小作为参数。  
初始化权重的更新代码看起来如下所示：

```python
self.wih=numpy.random.normal(0.0,pow(self.hnodes,-0.5),(self.hnodes,self.inodes))
self.who=numpy.random.normal(0.0,pow(self.onodes,-0.5),(self.hnodes,self.hnodes))
```

我们以正态分布的中心设定为0.0。与下一层节点相关的标准方差的表达式，按照Python的形式，就是pow(self.hnodes,-0.5),简单来说，这个表达式就是表示节点数目的-0.5次方。最后一个参数，就是我们希望的numpy数组的形状大小。

# 查询网络

接下来，顺理成章，我们现在应该编写神经网络的代码，填写当前空的train()函数。但是，还是等一下再写train()函数，让我们先编写简单的query()函数吧。这将会给我们更多的时间来逐步建立信心，获得试用Python和神经网络对象内部权重矩阵的实践经验。  
query()函数接受神经网络的输入，返回网络的输出。这个功能非常简单，但是，为了做到这一点，你要记住，我们需要传递来自输入层节点的输入信号，通过隐藏层，最后从输出层输出。你还要记住，当信号馈送至给定的萤囊层节点或输出层节点时，我们使用链接权重调节信号，还应用S激活函数抑制来自这些节点的信号。  
如果有很多节点那么我们就面临着一个很可怕的任务，即为这些节点中的每一个写出Python代码，进行权重调节，加和信号，应用激活函数。节点越多，代码越多。好在我们知道如何偷懒，下式显示了输入层和隐藏层之间的链接权重矩阵如何与输入矩阵相乘，给出隐藏层节点的输入信号。  
$$X_h=W_i \cdot I$$
这样做的好处不仅书写容易，python也可以理解矩阵，提高了工作效率。
```python
hidden_inputs=numpy.dot(self.wih,inputs)
```
这段代码完成了所有的工作，将所有的输入与所有正确的链接权重组合，生成了组合调节后的信号矩阵，传输给每个隐藏层节点。如果下一次选择使用不同数量的输入层节点或隐藏层几点，不必重写这段代码就可以进行工作。这种力量与优雅就是我们先前将精力投入到理解如何使用矩阵乘法的原因。  
为了获得从隐藏层节点处出现的信号，我们简单地将S已知函数应用到每一个出现的信号上：$$O_h=sigmoid(X_h)$$
如果在某个现成的Python库中，已经定义了这个S函数，那么这种操作就变得非常容易。果不其然！SciPy Python库有一组特殊的函数，在这组函数中，S函数称为expit()。
```python
import scipy,special
```

由于我们可能希望改变调整激活函数，在神经网络初始化部分的代码内部，下列代码定义了希望使用的激活函数。
```python
# activation function is the sigmoid function
self.actication_function=lambda x:scipy.special.expit(x)
```
在这里，就是创建一个函数，就像创建其他函数一样，我们不适用通常的def来定义函数，在此我们使用lambda来创建函数。这个函数接受了x，返回scipy.aspecial.expit(x)，这就是S函数。使用lambda创建的函数是没有名字的成为匿名函数。但是这里分给他一个名字，意味着，无论何时任何人使用激活函数，就是调用self.actication_function。
```python
# calculate the signals emerging from hidden layer
hidden_outputs=self.actication_function(hidden_inputs)
self.actication_function=lambda x:scipy.special.expit(x)
```

```python
# calculate sugnals into hidden layer
hidden_inputs=numpy.dot(self.wih,inputs)
# calculate sugnals emerging from hidden layer
hidden_outputs=self.activation_function(hidden_inputs)
# calculate sugnals into output layer
final_inputs=numpy.dot(self.who,hidden_outputs)
# calculate sugnals emerging from final output layer
final_outputs=self.activation_function(final_inputs)
```

# 迄今为止的代码
目前缺少train函数，请记住，在训练神经网络的过程中有两个阶段，第一个阶段就是计算输出，如同query所做的事情，第二个阶段就是反向传播误差，告知如何优化链接权重。  
在继续编写trian()函数并使用样本训练网络之前，让我们测试目前得到的所有的代码。我们创建一个小网络，使用一些随机输入查询网络，看看网络如何工作。显而易见，这样做不会有任何实际意义，只是为了使用刚刚创建的函数。

# 训练网络

**现在来解决这个稍微复杂的训练任务。训练任务分为两个部分：**
- 第一部分，针对给定的训练样本计算输出。这与我们刚刚在query（）函数早上所做的没什么区别
- 第二部分，将计算得到的输出与所需输出对比，使用差值来指导网络权重的更新。
我们已经完成了第一部分：
```python
def train(self,input_list,targets_list):
    # convert inputs list to 2d array
    inputs=numpy.array(inputs_list,ndmin=2).T
    targets=numpy.array(targets_list,ndmin=2).T
    # calculate sugnals into hidden layer
    hidden_inputs=numpy.dot(self.wih,inputs)
    # calculate sugnals emerging from hidden layer
    hidden_outputs=self.activation_function(hidden_inputs)
    # calculate sugnals into output layer
    final_inputs=numpy.dot(self.who,hidden_outputs)
    # calculate sugnals emerging from final output layer
    final_outputs=self.activation_function(final_inputs)
    pass
```

现在我们越来越接近神经网络工作的核心，即基于所计算输出与目标输出之间的误差，改进权重。  
让我们按照轻柔可控的步骤，进行这种操作。  
首先需要计算误差，这个值等于训练样本所提供的预期目标输出至与实际计算得到的输出值之差。这个差也就是将矩阵targets和矩阵final_outputs中每个对应元素相减得到的。Python代码非常简单，这再次优雅地显示了矩阵的力量。
```python
# error is the (target-actual)
output_errors=targets-final_outputs
```

我们可以计算出隐含层节点反向传播的误差。$$errors_{hidden}=weights^T_{hidden\_output} \cdot errors_{output}$$

```python
# hidden layer error is the output_errors,split by weights, recombined at hidden nodes
hidden_errors=numpy.dot(self.who.T,output_errors)
```

**这样我们就拥有了所需要的一切，可以优化各个层之间的权重了。对于在隐蔽层和最终层之间的权重，我们是用output_errors进行优化。对于输入层和隐藏层之间的权重，我们使用钢材计算得到的hidden_errors进行优化。**
$$\delta W_{j,k}= \alpha * E_k*sigmoid(O_k)*(1-sigmoid(O_k)) \cdot O_j^T$$
其中α是学习率，s是激活函数，在Python代码中，这种转换很容易。我们首先为隐藏层和最终层之间的权重进行编码
```python
# update the weights for the links between the hidden and output layers
self.who+=self.lr*numpy.dot((output_errors*final_outputs*(1.0-final_outputs)),numpy.transpose(hidden_outputs))
# update the weights for the links between the input and hidden layers
self.wih+=self.lr*numpy.dot((hidden_errors*hidden_outputs*(1.0-hidden_outputs)),numpy.transpose(inputs))
```